1.ii

In [17]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import dh, dsa
from cryptography.hazmat.primitives import serialization, hashes
from getpass import getpass
import getpass, os, io
from cryptography.exceptions import *

default_algorithm = hashes.SHA256

# Gerar os parâmetros

parameters = dh.generate_parameters(generator=2, key_size=1024,
                                     backend=default_backend())

parameters_dsa = dsa.generate_parameters(key_size=1024, backend=default_backend())

def Hash(s):
    digest = hashes.Hash(default_algorithm(),backend=default_backend())
    digest.update(s)
    return digest.finalize()

In [18]:
def Alice(conn):
    # agreement
    pk = parameters.generate_private_key()
    pub = pk.public_key().public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo)
    conn.send(pub)

    
    # shared_key calculation
    peer_pub_key = serialization.load_pem_public_key(
            conn.recv(),
            backend=default_backend())
    shared_key = pk.exchange(peer_pub_key)
    
    # confirmation
    my_tag = Hash(bytes(shared_key))
    conn.send(my_tag)
    peer_tag = conn.recv()
    if my_tag == peer_tag:
        print('OK')
    else:
        print('FAIL')

    #ASSINAR
    
   #gerar as chaves privada e pública
    private_key_dsa = parameters_dsa.generate_private_key()
    pub_dsa = private_key_dsa.public_key().public_bytes( 
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo)
    
    #envia a chave pública
    conn.send(pub_dsa)
    
    #cálculo da assinatura
    signature = sig = private_key_dsa.sign(pub, hashes.SHA256())
   
    peer_pub_dsa =serialization.load_pem_public_key( #vou deserializa-la
        conn.recv(), #agora recebo o a^y
        backend=default_backend())  
    
    
    conn.send(signature)
    
    
    #verificar
    
    
    try:
        peer = conn.recv()
        peer_pub_dsa.verify = (peer, peer_pub_key, hashes.SHA256())
        print("ok")
    except InvalidSignature:
        print("fail")

In [19]:
import os
from getpass import getpass
from multiprocessing import Process, Pipe

    
class BiConn(object):
    def __init__(self,left,right,timeout=None):
        """
        left : a função que vei ligar ao lado esquerdo do Pipe
        right: a função que vai ligar ao outro lado
        timeout: (opcional) numero de segundos que aguarda pela terminação do processo
        """
        left_end, right_end = Pipe()
        self.timeout=timeout
        self.lproc = Process(target=left, args=(left_end,))       # os processos ligados ao Pipe
        self.rproc = Process(target=right, args=(right_end,))
        self.left  = lambda : left(left_end)                       # as funções ligadas já ao Pipe
        self.right = lambda : right(right_end)
    
    def auto(self, proc=None):
        if proc == None:             # corre os dois processos independentes
            self.lproc.start()
            self.rproc.start()  
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:                        # corre só o processo passado como parâmetro
            proc.start(); proc.join()
    
    def manual(self):   #  corre as duas funções no contexto de um mesmo processo Python
        self.left()
        self.right()

In [20]:
BiConn(Alice,Alice).auto()

OK
OK
ok
ok
